In [ ]:
import sys
import os

sys.path.append('../charting')
sys.path.append('../run_analysis')

from support.util import titles
from support.charting import primaryColors, fullColors, save_fig
from modelConfig import models

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker  
import numpy.lib.recfunctions as rf
import seaborn as sns

import numpy as np
import pandas as pd
import random
import copy

from ema_workbench import (perform_experiments, ema_logging, save_results, 
                           load_results)
from ema_workbench.em_framework import samplers
from ema_workbench.em_framework import sample_levers

# turn on logging
ema_logging.log_to_stderr(ema_logging.INFO)

In [ ]:
# Setup and Plotting Methods

fs = 12
def selectedValueRanges(results, selectType): 
    # Display value ranges of the identified vulnerable scenarios 
    dfs = []

    df = pd.DataFrame(results['dps'][0])
    dfs.append(df[['b','q','mean','stdev','delta','model']])

    df = pd.DataFrame(results['plannedadaptive'][0])
    df['model'] = 'plannedadaptive'
    dfs.append(df[['b','q','mean','stdev','delta','model']])

    df = pd.DataFrame(results['intertemporal'][0])
    dfs.append(df[['b','q','mean','stdev','delta','model']])

    uncs = pd.concat(dfs)

    vals = []
    for model in ['intertemporal','plannedadaptive','dps']:
        d = uncs.loc[uncs['model']==model]
        for unc in ['b','q','mean','stdev','delta']: 

            df = pd.DataFrame(d[unc])
            df.columns = ['value']
            df['model'] = model
            df['uncertainty'] = unc
            vals.append(df)
    uncs = pd.concat(vals)

    sns.set_style('white')
    pal = {'intertemporal': primaryColors['intertemporal']['multi'],
           'plannedadaptive': primaryColors['plannedadaptive']['multi'],
           'dps': primaryColors['dps']['multi']}
    for tp in ['box','violin']: 
        g = sns.factorplot(x="model", y="value", col="uncertainty", palette=pal,
                           data=uncs, kind=tp, sharey=False,sharex=True,
                           size=4, aspect=.7)
        g.set_axis_labels('', "Value").set_titles("{col_name}").set_xticklabels(['Inter.','Planned\nAdaptive','DPS'])

        plt.suptitle('Scenario Selection Uncertainty Value Ranges', fontsize=18,weight='bold',y=1.05)

        save_fig(g, 'images/', selectType + '_leverranges_' + tp + '_' + 'scens')

        plt.show()
                
class Character(object):
    '''
    returns a character in the order of the alphabet    
    '''

    def __init__(self):
        self.counter = -1
        
    def __call__(self, kwargs):
        self.counter += 1
        if self.counter == 26: 
            self.counter = 0
        return chr(ord('a') + self.counter)

# Generate Ensemble of Experiments 

The inputs and outputs from which to select 4 reference scenarios. Settings are at the top of the box. The larger the ensemble, the more computing time required to find a maximally diverse set. 

Settings used here were used in the thesis that incorporated this work. 

This code checks for existing data that was already generated and loads it from a file wherever possible. 

In [ ]:
# perform experiments
nr_experiments = 500
nr_policies = 10
num_scenarios = 4
folder = '../data/multi/scenarioselection/scens_pol' + str(nr_policies) + '/'

results = {}
for modelName, model in models.items(): 
    if not os.path.exists(folder):
        os.makedirs(folder)

    fn = '{}{}_{}experiments_{}policies.tar.gz'.format(folder, modelName, nr_experiments, nr_policies)
    policies = sample_levers(model,  n_samples=nr_policies, name=Character())

    try:
        # why regenerate the data?
        results[modelName] = load_results(fn)
    except IOError:
        results[modelName] = perform_experiments(model, nr_experiments, policies=policies)
        save_results(results[modelName], fn)

## Select relevant results based on means

Select input settings from the set of all experiments generated based on performance in the lower half for all outcomes of interest. This is just one option for selecting reference scenarios. 

The thesis uses primarily results selection based on scenario discovery analysis, which is shown later in the notebook. 


In [ ]:
newResults = {}
logicalIndex = {}

for key, model in models.items(): 
    experiments, outcomes = results[key]
    #here, the policy-relevant scenarios defined by median thresholds are selected
    indices = []
    for outcome in model.outcomes:
        if outcome.kind == -1:
            a = outcomes[outcome.name] > np.mean(outcomes[outcome.name])     
        else: 
            a = outcomes[outcome.name] < np.mean(outcomes[outcome.name])
        indices.append(a)
    indices = np.swapaxes(indices, 0, 1)
    logical_index = np.array([index.all() for index in indices])
    logicalIndex[key] = logical_index

    new_fn = '{}{}_meanselected_{}experiments_{}policies.tar.gz'.format(folder, key, nr_experiments, nr_policies)
    try:
        # why regenerate the data?
        newResults[key] = load_results(new_fn)
        print(key, newResults[key][0].shape)

    except IOError:
        newExperiments = experiments[logical_index]
        newOutcomes = {}
        for outcome in model.outcomes:
            newOutcomes[outcome.name] = outcomes[outcome.name][logical_index]

        newResults[key] = newExperiments, newOutcomes
        print(newResults[key][0].shape)
        
# Display value ranges of the identified vulnerable scenarios 
selectedValueRanges(newResults, 'mean')

## Select relevant results based on median

In [ ]:
newResults = {}
logicalIndex = {}

for key, model in models.items(): 
    experiments, outcomes = results[key]
    #here, the policy-relevant scenarios defined by median thresholds are selected
    indices = []
    for outcome in model.outcomes:
        if outcome.kind == -1:
            a = outcomes[outcome.name] > np.median(outcomes[outcome.name])     
        else: 
            a = outcomes[outcome.name] < np.median(outcomes[outcome.name])
        indices.append(a)
    indices = np.swapaxes(indices, 0, 1)
    logical_index = np.array([index.all() for index in indices])
    logicalIndex[key] = logical_index

    new_fn = '{}{}_medianselected_{}experiments_{}policies.tar.gz'.format(folder, key, nr_experiments, nr_policies)
    try:
        # why regenerate the data?
        newResults[key] = load_results(new_fn)
        print(key, newResults[key][0].shape)

    except IOError:
        newExperiments = experiments[logical_index]
        newOutcomes = {}
        for outcome in model.outcomes:
            newOutcomes[outcome.name] = outcomes[outcome.name][logical_index]

        newResults[key] = newExperiments, newOutcomes
        save_results(newResults[key], new_fn)

        print(key)
        print(newResults[key][0].shape)

# Display value ranges of the identified vulnerable scenarios 
selectedValueRanges(newResults, 'mean')

## Select relevant results based on Prim results

Select input settings from the set of all experiments generated based on Prim results, as done in this thesis.

In [ ]:
prim_box = {'dps': {'b' : (0.100370, 0.239250), 
                    'delta': (0.930044, 0.959158),
                    'q': (2.001502, 4.145256), 
                    'c1': (-0.104005, 1.731693)}, 
            'plannedadaptive': {'b' : (0.100637, 0.402237), 
                               'delta': (0.930046,0.944981)},
            'intertemporal': {'b' : (0.100240, 0.261456), 
                              'delta': (0.930061,0.959012), 
                              'q' : (2.001704, 4.207757),
                              'l1' : (0.006307, 0.088614), 
                              'l11' : (0.001967, 0.088614),
                              'l12' : (0.000645, 0.088072),
                              'l13' : (0.008911, 0.091266)}
           }


In [ ]:
primIndex = {}
primResults = {}
for modelkey, model in models.items(): 
    print(modelkey)
    experiments, outcomes = results[modelkey]
    
    prim_indices = []
    for key, value in prim_box[modelkey].items():  
        a_ = experiments[key] >= value[0]
        b_ = experiments[key] <= value[1]
        c = [np.array([a, b]).all() for a, b in zip(a_, b_)]
        prim_indices.append(c)

    prim_indices = np.swapaxes(prim_indices, 0, 1)
    prim_logical_index = np.array([index.all() for index in prim_indices])
    
    newExperiments = experiments[prim_logical_index]
    newOutcomes = {}
    for outcome in model.outcomes:
        newOutcomes[outcome.name] = outcomes[outcome.name][logical_index]
        
    print(newExperiments.shape)
    primResults[modelkey] = newExperiments, newOutcomes
    primIndex[modelkey] = prim_logical_index

    new_fn = '{}{}_primselected_{}experiments_{}policies.tar.gz'.format(folder, modelkey, nr_experiments, nr_policies)
    save_results(primResults[modelkey], new_fn)
    
# Display value ranges of the identified vulnerable scenarios 
selectedValueRanges(newResults, 'mean')

### Finding the maximally diverse set

Due to large computational constraints, the maximally diverse set of alternatives is found using a Python script outside the Jupyter notebook environment. The script is called `MaxDiverseSeelct.py` in the same folder as this notebook. 

To run this script and find the set of maximally diverse scenarios, follow these instructions. The script requires four command line parameters, passed in the order described. 

1. model: The model name (one of 'intertemporal', 'plannedadaptive', and 'dps' in this case). 
2. selection type: what selection method was used ('prim' in this case). 
3. number of policies: the number of policies used to generate experiment data (10 in this case). 
4. number of experiments: the number of experiments used to generate experiment data (500 in this case). 
5. set size: the number of reference scenarios to identifiy (4 in this case). 

These four parameters lead to the following command, for example: 

```shell
python MaxDiverseSelect.py dps prim 10 500
```

This script is threaded to improve run time, with the number of threads equal to the cpu count. There is no hard limit to the number of threads because each diversity calculation is independent. The results of each thread are simply compared to each other at the end of execution to find the single set of maximally diverse alternatives. 